In [ ]:
import os
import sys
import numpy as np
from numpy.linalg import inv
if '..' not in sys.path:
    sys.path = ['..'] + sys.path
from pfcommon import parse_Jacobian_vars_file, parse_sparse_matrix_file
from filter_OU_inputs import run_welch

In [ ]:
import seaborn as sns
import matplotlib
import matplotlib.pyplot as plt
fontsize = 9
lw = 0.75
matplotlib.rc('font', **{'family': 'Arial', 'size': fontsize})
matplotlib.rc('axes', **{'linewidth': 0.75, 'labelsize': fontsize})
matplotlib.rc('xtick', **{'labelsize': fontsize})
matplotlib.rc('ytick', **{'labelsize': fontsize})
matplotlib.rc('xtick.major', **{'width': lw, 'size': 3})
matplotlib.rc('ytick.major', **{'width': lw, 'size': 3})
matplotlib.rc('ytick.minor', **{'width': lw, 'size': 1.5})

FIGURES_DIR = 'figures'

In [ ]:
folder = '../data/SM_with_load/adynamic_load_const_S/LD1'
folder = '../data/SM_with_line_and_load/adynamic_load_const_S/LD1'
folder = '../data/SM_with_line_and_loads/adynamic_load_const_S/LD1'
Amat = parse_sparse_matrix_file(os.path.join(folder, 'Amat.mtl'))
N_state_vars = Amat.shape[0]
J = parse_sparse_matrix_file(os.path.join(folder, 'Jacobian.mtl'))
N_vars = J.shape[0]
N_algebraic_vars = N_vars - N_state_vars
print('# of state variables: {}'.format(N_state_vars))
print('# of algebraic variables: {}'.format(N_algebraic_vars))

In [ ]:
vars_idx, state_vars, voltages, currents, signals = parse_Jacobian_vars_file(os.path.join(folder, 'VariableToIdx_Jacobian.txt'))
element_names = {'gen': 'ElmSym', 'bus': 'ElmTerm', 'load': 'ElmLod'}
# generator speed
device_group = 'gen'
device_name = 'G1'
var_name = 's:speed'
# var_name = 'm:ii:bus1'
# bus voltage
# device_group = 'bus'
# device_name = 'Bus2'
# var_name = 'm:ur'
# load current
# device_group = 'load'
# device_name = 'LD2'
# var_name = 'm:ir:bus1'
output_var_idx = vars_idx['Grid-{}.{}'.format(device_name, element_names[device_group])][var_name.split(':')[1]][0]

In [ ]:
Jfx = J[:N_state_vars, :N_state_vars]
Jfy = J[:N_state_vars, N_state_vars:]
Jgx = J[N_state_vars:, :N_state_vars]
Jgy = J[N_state_vars:, N_state_vars:]

print('Shape of Jfx: {}'.format(Jfx.shape))
print('Shape of Jfy: {}'.format(Jfy.shape))
print('Shape of Jgx: {}'.format(Jgx.shape))
print('Shape of Jgy: {}'.format(Jgy.shape))

In [ ]:
Jgy_inv = inv(Jgy)
A = Jfx - np.dot(np.dot(Jfy, Jgy_inv), Jgx)
assert np.allclose(A, Amat), 'Error in the computation of the matrix A'
B = - np.dot(Jfy, Jgy_inv)
C = - np.dot(Jgy_inv, Jgx)
D = - Jgy_inv
print('Shape of A: {}'.format(A.shape))
print('Shape of B: {}'.format(B.shape))
print('Shape of C: {}'.format(C.shape))
print('Shape of D: {}'.format(D.shape))

In [ ]:
load_type = 'stochastic'
tran_filename = 'SM_with_load_tran_0.01'
tran_filename = 'SM_with_line_and_load_tran_0.01'
tran_filename = 'SM_with_line_and_loads_tran_0.01'
tran_data = np.load(os.path.join(folder, tran_filename + '.npz'), allow_pickle=True)
PF = tran_data['PF_without_slack'].item()
t_tran = tran_data['time']
dt_tran = t_tran[1] - t_tran[0]
data = tran_data['data'].item()
tran_start = 100 # [s]
device_names = tran_data['device_names'].item()
idx = device_names[device_group].index(device_name)
if data[device_group][var_name].ndim == 1:
    assert idx == 0
    x_tran = data[device_group][var_name][t_tran > tran_start]
else:
    x_tran = data[device_group][var_name][t_tran > tran_start, idx]

Δx_tran = (x_tran - x_tran.mean())
freq_tran, P_tran, abs_tran = run_welch(Δx_tran, dt_tran, window=200/dt_tran, onesided=True)

In [ ]:
I = np.eye(N_state_vars)
Fmin, Fmax = -6, 2
steps_per_decade = 100
F = np.logspace(Fmin, Fmax, (Fmax - Fmin) * steps_per_decade + 1)
input_loads = 'LD1',
N_input_loads = len(input_loads)
TF = np.zeros((F.size, N_input_loads, N_vars), dtype=complex)
OUT = np.zeros((F.size, N_input_loads, N_vars), dtype=complex)

input_rows = {ld: np.zeros(2, dtype=int) for ld in input_loads}
coeffs = {ld: np.zeros(2, dtype=float) for ld in input_loads}
for ld in input_loads:
    bus_name = 'Bus1' # this must be the name of the bus ld is connected to
    ur, ui = PF['buses'][bus_name]['ur'], PF['buses'][bus_name]['ui']
    den = np.abs(ur + 1j * ui) ** 2
    for i, suffix in enumerate('ri'):
        cols = vars_idx['Grid-{}.ElmLod'.format(ld)]['i' + suffix]
        assert len(cols) == 1
        col = cols[0]
        input_rows[ld][i] = int(np.argmin(np.abs(J[:, col] - (-1))))
        coeffs[ld][i] = PF['buses'][bus_name][f'u{suffix}'] / den
        print("Variable 'i{}' of object '{}' is at column {}: equation #{}.".\
            format(suffix, ld, col + 1, input_rows[ld][i] + 1))

τ = 20e-3
μ = tran_data['stoch_load_P']
dP = 0.01
σ = np.array([m * dP for m in μ])
α = np.array([1 / τ for _ in range(N_input_loads)])
c = np.array([s * np.sqrt(2 / τ) for s in σ])
for i, f in enumerate(F):
    M = 1j * 2 * np.pi * f * I - A # sI - A
    MinvxB = np.dot(inv(M), B)            # (sI - A)^-1 x B
    for j, ld in enumerate(input_loads):
        psd = np.sqrt((c[j] / α[j])**2 / (1 + (2 * np.pi * F[i] / α[j])**2))
        v = np.zeros(N_algebraic_vars)
        v[input_rows[ld] - N_state_vars] = 1.
        v[input_rows[ld] - N_state_vars] *= coeffs[ld]
        TF[i, j, :N_state_vars] = np.dot(MinvxB, v)
        TF[i, j, N_state_vars:] = np.dot(np.dot(C, MinvxB) + D, v)
        v[input_rows[ld] - N_state_vars] = psd
        v[input_rows[ld] - N_state_vars] *= coeffs[ld]
        OUT[i, j, :N_state_vars] = np.dot(MinvxB, v)
        OUT[i, j, N_state_vars:] = np.dot(np.dot(C, MinvxB) + D, v)

In [ ]:
dB = 10
input_var_idx = 0
Y = OUT[:, input_var_idx, output_var_idx]
y = dB * np.log10(np.abs(Y)) if dB in (10, 20) else np.abs(Y)

fig,ax = plt.subplots(2, 1, figsize=(5,5))
ax[0].plot(t_tran[t_tran > tran_start], x_tran, 'k', lw=1)
ax[0].plot([tran_start, t_tran[-1]], x_tran.mean() + np.zeros(2), 'r--', lw=1.5)
ax[0].set_xlabel('Time [s]')
ax[0].set_ylabel('{} {} [p.u.]'.format(var_name.split(':')[1], device_group))
ax[1].plot(freq_tran, dB * np.log10(abs_tran) if dB in (10, 20) else abs_tran, 'k', lw=1)
ax[1].plot(F, y, 'tab:red', lw=2)
if device_group == 'load' and 'loads' not in folder:
    # 10 * np.log10(I_base_gen / I_base_load)
    offset = 2.9251332576568743
    ax[1].plot(F, y + offset, 'tab:green', lw=2)
ax[1].set_xscale('log')
ax[1].set_xlabel('Frequency [Hz]')
ax[1].set_ylabel('|Y(jω)|')
ax[1].set_xlim([1e-3, 10**Fmax])
sns.despine()
fig.tight_layout()
outfile = 'PSD_PF_{}_{}.pdf'.format(var_name.split(':')[1], device_group)
# plt.savefig(os.path.join(FIGURES_DIR, outfile))